In [54]:
from tai_chi_tuna.config import PhaseConfig
from tai_chi_tuna.front.html import DOM, list_group_kv
from pathlib import Path

In [18]:
!mkdir -p example1

In [25]:
%%writefile example1/phase.json
{
  "enrich": [
    {
      "src": "image_path",
      "dst": "image",
      "kwargs": {
        "convert": "RGB",
        "size": 224
      },
      "enrich": "EnrichImage"
    }
  ],
  "quantify": [
    {
      "src": "image",
      "x": true,
      "kwargs": {
        "mean_": "imagenet",
        "std_": "imagenet"
      },
      "quantify": "QuantifyImage"
    },
    {
      "src": "Genre",
      "x": false,
      "kwargs": {
        "min_frequency": 1,
        "separator": ";"
      },
      "quantify": "QuantifyMultiCategory"
    }
  ],
  "batch_level": {
    "valid_ratio": 0.1,
    "batch_size": 32,
    "shuffle": true,
    "num_workers": 0
  },
  "x_models": {
    "image": {
      "model_name": "ImageConvEncoder",
      "src": "image",
      "kwargs": {
        "name": "resnet34"
      }
    }
  },
  "y_models": {
    "Genre": {
      "model_name": "MultiCategoryTop",
      "src": "Genre",
      "kwargs": {
        "hidden_size": 0
      }
    }
  },
  "task_slug": "image_to_Genre"
}

Overwriting example1/phase.json


In [26]:
phase = PhaseConfig.load(Path("./example1"))

In [120]:
def join_col_list(Columns):
    return ",".join(map(lambda x:f"'{x['src']}'", Columns))

In [123]:
class Narrator:
    x_color = "#3399FF"
    y_color = "#FF5533"
    def __init__(self, phase):
        self.phase=phase
        
    def __call__(self,):
        self.narrate()()
        
    def title(self, text) -> DOM:
        return DOM(text, "h3",
                   {"style":"color:#FFAA33;background-color:#FFFFFF;padding:3px"})
    
    def empty_step(self,):
        return DOM("","ul",dict())
    
    def new_line(self, inner):
        return DOM(inner,"li",dict(style="padding:2px;background-color:#FFFFDD"))
        
    def narrate(self,):
        config = self.phase.config
        doc = DOM("","div")
        step =  self.empty_step()
        step.append(self.title("Step 1: Enrich"))
        if "enrich" in config:
            enrichs = config['enrich']
            for enrich in enrichs:
                src = enrich.get('src')
                enrich_type = enrich.get('enrich')
                step.append(self.new_line(f"select column '{src}' for 'src'"))
                step.append(
                    self.new_line(
                        f"select '{enrich_type}' for 'enrich'"))
                step.append(
                    self.new_line(
                        f"The kwargs of '{enrich_type}' should be set to:"))
                step.append(self.new_line(list_group_kv(enrich.get("kwargs"))))
            step.append(self.new_line("Congrats! now Click ✔️Next above"))
        else:
            step.append(self.new_line("No enrich set, click ✔️Next above"))
        doc.append(step)
        
        step =  self.empty_step()
        step.append(self.title("Step 2: Quantify"))
        if "quantify" in config:
            quantifies  = config['quantify']
            Xs = list(filter(lambda x:x['x'], quantifies))
            Ys = list(filter(lambda x:x['x']==False, quantifies))
            
            step.append(
                self.new_line(f"""
                In this model we are using 
                <strong style='color:{self.x_color}'>
                column(s):{join_col_list(Xs)}</strong> 
                to guess <strong style='color:{self.y_color}'>
                column:{join_col_list(Ys)}</strong>
                """)
            )
            
            for is_x, Columns in zip([True, False],[Xs,Ys]):
                if is_x:
                    step.append(
                        self.new_line(
                            f"""
                            <h4 style='color:{self.x_color}'>
                            There are {len(Xs)} X column(s):</h4>"""
                        ))
                else:
                    step.append(
                        self.new_line(
                            f"""
                            <h4 style='color:{self.y_color}'>
                            There is our {len(Ys)} Y column:</h4>"""
                        ))
                for col in Columns:
                    step.append(
                        self.new_line(f"""
                        <h5>Quantify config for '{col['src']}' column</h5>""")
                    )
                    step.append(
                        self.new_line(f"""
                        Select '{col['src']}' as 'src'"""))
                    step.append(
                        self.new_line(f"""
                        Choose 'As {'X' if is_x else 'Y'}', 
                        So column {col['src']} is used as 
                        {'input' if is_x else 'output'} data"""))
                    step.append(
                        self.new_line(f"Click Button: 'Run Interact'"))
                    step.append(
                        self.new_line(f"""
                        Choose '{col['quantify']}' as the method to 
                        transform '{col['src']}' into matrix/tensor"""))
                    if 'kwargs' in col:
                        step.append(
                            self.new_line(
                                f"Click Button: 'Run Interact' to set kwargs"))
                        step.append(
                            self.new_line(
                                f"The kwargs of '{col['quantify']}' should be set to:"))
                        step.append(
                            self.new_line(list_group_kv(col["kwargs"])))
                        step.append(
                            self.new_line(f'Click Button:+Create'))
                    else:
                        step.append(
                            self.new_line(f"""
                            Click Button: 'Run Interact' then click '+Create'"""))
            step.append(self.new_line(
                "Congrats! now Click ✔️Next above"))
        doc.append(step)
        return doc

In [124]:
Narrator(phase)()

NameError: name 'x_color' is not defined